In [5]:
pip install keyboard


In [ ]:
import cv2
import numpy as np
import os
import serial
import time
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model(r'C:\Users\lenovo\Desktop\custom_vgg16_model.h5')

# Define the folder path to save the images
folder_path = r'E:\Trash'

# Ensure the folder is empty before starting
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

# Define constants
IMG_SIZE = 32
CATEGORIES = ['can', 'glass', 'paper', 'plastic']
class_labels = {0: "can", 1: "glass", 2: "paper", 3: "plastic"}

# Function to capture image and save to folder
def capture_image():
    cap = cv2.VideoCapture(1)  # Use camera ID (adjust if needed)
    
    if not cap.isOpened():
        print("Cannot open camera")
        return None
    
    # Capture an image
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return None

    # Save the captured image
    image_path = os.path.join(folder_path, 'captured_image.jpg')
    cv2.imwrite(image_path, frame)

    cap.release()
    cv2.destroyAllWindows()

    return image_path

# Function to preprocess the image and classify
def classify_image(image_path):
    try:
        # Load and preprocess the image
        img_array = cv2.imread(image_path)
        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        img_array = img_array / 255.0  # Normalize the image

        # Expand dimensions to match the model input
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        prediction = model.predict(img_array).argmax(axis=1)[0]
        trash_type = class_labels[prediction]
        return trash_type
    except Exception as e:
        print(f"Error in classify_image: {e}")
        return None

# Function to clean up folder
def clean_folder():
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Initialize serial connection to Arduino
try:
    arduino = serial.Serial(port='COM3', baudrate=9600, timeout=1)  # Adjust 'COM3' to match your port
    time.sleep(2)  # Allow some time for Arduino to establish connection
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    exit()

# Main loop to handle serial communication and process images
try:
    while True:
        if arduino.in_waiting > 0:
            # Read data from Arduino
            data = arduino.readline().decode('utf-8').strip()

            print(f"Received from Arduino: {data}")  # Debugging: show received data

            if data == "close":  # If Arduino sends "close" signal
                print("Arduino sent 'close', capturing image...")

                # Capture image
                image_path = capture_image()
                if image_path:
                    # Classify the image
                    trash_type = classify_image(image_path)
                    if trash_type:
                        print(f"Classified as: {trash_type}")

                        # Send classification back to Arduino
                        arduino.write(f"{trash_type}\n".encode())  # Ensure newline for Arduino
                        time.sleep(1)  # Allow some time for Arduino to process

                        # Clean up the folder (delete the image)
                        clean_folder()
                        print("Image deleted, folder cleaned up.")
                    else:
                        print("Failed to classify image.")
                else:
                    print("Failed to capture image.")

            elif data == "open":  # If Arduino sends "open" signal, stop the process
                print("Arduino sent 'open', stopping the process...")
                break  # Exit the loop when "open" is received
            
            # Option to quit by pressing 'q' (optional)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

except KeyboardInterrupt:
    print("Program interrupted by user.")
finally:
    if 'arduino' in locals() and arduino.is_open:
        arduino.close()
        print("Serial connection closed.")

Received from Arduino: close
Arduino sent 'close', capturing image...
Classified as: can
Image deleted, folder cleaned up.
Received from Arduino: open
Received from Arduino: close
Arduino sent 'close', capturing image...
Classified as: paper
Image deleted, folder cleaned up.
Serial connection closed.


SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))